In [ ]:
import os
import pandas as pd
from emobpy import DataBase
from emobpy import Mobility, DataBase, Consumption, HeatInsulation, BEVspecs, Availability, Charging
import shutil

: 

In [ ]:
home_path = os.getcwd()
os.makedirs(os.path.join(home_path, "db", "used"), exist_ok=True)
consumption_path = os.path.join(home_path, "db", "consumption")
availability_path = os.path.join(home_path, "db", "availability")
profile_path  = os.path.join(home_path, "db", "profile")

DB_profile = DataBase('db/profile')

#os.chdir("../")
data_path = os.getcwd()

: 

In [ ]:
number_profiles = 1
year = 2020


: 

In [ ]:
def get_dir_size(path='.'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            #elif entry.is_dir():
             #   total += get_dir_size(entry.path)
    total = total / (10**9)
    return total

def create_new_folder(path, profile_consumption_availability = None, used_dir=False):
    if used_dir == False:
        folder_list = [d for d in os.listdir(path)
                   if os.path.isdir(os.path.join(path, d))]
    
        if len(folder_list) == 0:
            new_f = "0"   
        else:
            new_f = str(max(map(int,folder_list)) + 1)
        new_f_path = os.path.join(path, new_f)
        
    else: 
        new_f_path = os.path.join(home_path, "db", "used", profile_consumption_availability)
    
    os.makedirs(new_f_path, exist_ok=True)

    return new_f_path

def transtion_files(source_dir):
    target_dir = create_new_folder(source_dir)

    file_names = [f for f in os.listdir(source_dir)
                       if os.path.isfile(os.path.join(source_dir, f))]

    for file_name in file_names:
        shutil.move(os.path.join(source_dir, file_name), target_dir)
        
def get_folders(path): 
    f_list = [d for d in os.listdir(path)
              if os.path.isdir(os.path.join(path, d))]
    if ".DS_Store" in f_list:
        f_list.remove(".DS_Store")
    
    return f_list

: 

## Step 1: Vehicle mobility time series

In [ ]:
from emobpy import Mobility

def mobility_gen (i):
    m = Mobility(config_folder='config_files')
    m.set_params(
             name_prefix="BEV{}".format(i),
             total_hours=2160, 
             time_step_in_hrs=0.5, # 15 minutes
             category="user_defined",
             reference_date=f"01/01/{year}"
            )
    m.set_stats(
            stat_ntrip_path="TripsPerDay.csv",
            stat_dest_path="DepartureDestinationTrip.csv",
            stat_km_duration_path="DistanceDurationTrip.csv",
            )
    m.set_rules(rule_key="user_defined") 
    # see /config_files/rules.yml, it contains a dictionary, whose key must be the same
    # as rule_key. To see all possible rules `from emobpy.constants import RULE; RULE` 
    m.run()
    m.save_profile(folder="db/profile", description='') # save all info in pickle file in 'db' folder

: 

In [ ]:
for i in range(number_profiles):
    
    if get_dir_size(profile_path) >= 1:
        create_new_folder(profile_path)
        transtion_files(profile_path)
        
        
    try: 
        mobility_gen(i)
    except ValueError:
        pass
    
    DB_profile.update()                # This load new files hosted in database folder as result of new generated files

transtion_files(profile_path)

: 

In [ ]:
#m.kind
#m.description
#m.timeseries
#m.profile.head()
#from emobpy.plot import NBplot
#fig_mobility = NBplot(DB_profile).sgplot_dp(m.name)
#fig_mobility

: 

------------------------------------------------------
## Step 2: Driving consumption time series
------------------------------------------------------
a) Vehicle model configuration

------------------------------------------------------

In [ ]:
from emobpy import Consumption, HeatInsulation, BEVspecs

: 

In [ ]:
HI = HeatInsulation(True)            # Creating the heat insulation by copying the default configuration
BEVS = BEVspecs()

models = {
    "Tesla": 
        {"model": "Model 3 Long Range AWD",
         "year": 2019,
         "share": 22.73},
    "Volkswagen":
        {"model": "ID.3",
         "year": 2020,
         "share": 15.91},
    "Renault":
        {"model": "Zoe Q90",
         "year": 2017,
         "share": 15.91},
    "Hyundai":
        {"model": "Kona Electric 64 kWh",
         "year": 2019,
         "share": 11.63},
    "FIAT":
        {"model": "500e",
         "year": 2019,
         "share": 9.09},
    "BMW":
        {"model": "i3 33 kWh",
         "year": 2017,
         "share": 6.82},
    "MINI":
        {"model": "Cooper SE Level I",
         "year": 2020,
         "share": 6.82},
    "Audi": 
        {"model": "e-tron 55 quattro",
         "year": 2019,
         "share": 4.55},
    "Hyundai": 
        {"model": "IONIQ Electric 28 kWh",
         "year": 2019,
         "share": 4.55},
    "KIA": 
        {"model": "e-Niro 64 kWh",
         "year": 2019,
         "share": 2.27}
    
}

back_up_model = {
    "brand": "Volkswagen",
    "model": "e-Golf SE",
    "year": 2019
}

: 

In [ ]:
p_list = os.listdir(profile_path)
if ".DS_Store" in p_list:
    p_list.remove(".DS_Store")
elif '.ipynb_checkpoints' in p_list:
    p_list.remove('.ipynb_checkpoints')
    
number_profiles_all = -1

for folder in p_list:
    number_profiles_all += len([d for d in os.listdir(os.path.join(profile_path, folder))
     if os.path.isfile(os.path.join(profile_path, folder, d))])
    
models_assigned = {}

i = 0
for brand in models.keys():
    models[brand]["number"] = round(number_profiles_all*models[brand]["share"]/100)
    #print(brand + " " +str(models[brand]["number"]))
    for k in range(0, round(models[brand]["number"])): 
        print(str(i))
        models_assigned[str(i)] = BEVS.model((
            brand, models[brand]["model"], models[brand]["year"]
        ))
        i += 1
        
while i <= number_profiles_all:
    
    models_assigned[str(i)] = BEVS.model((
            back_up_model["brand"], back_up_model["model"], back_up_model["year"]
        ))
    i += 1

: 

In [ ]:
#plausibility check
len(models_assigned) - number_profiles_all

: 

In [ ]:
def consumption_gen(i, file, DB):
    car_model = models_assigned[str(i)]
    print(car_model.parameters['EV Model'])
    c = Consumption(file, car_model)
    c.load_setting_mobility(DB)
    try:
        c.run(
            heat_insulation=HI,
            weather_country='DE',
            weather_year=year,
            passenger_mass=75,                   # kg
            passenger_sensible_heat=70,          # W
            passenger_nr=1.5,                    # Passengers per vehicle including driver
            air_cabin_heat_transfer_coef=20,     # W/(m2K). Interior walls
            air_flow = 0.02,                     # m3/s. Ventilation
            driving_cycle_type='WLTC',           # Two options "WLTC" or "EPA"
            road_type=0,                         # For rolling resistance, Zero represents a new road.
            road_slope=0
        )
        c.save_profile(consumption_path)
    except ValueError:
        pass

: 

In [ ]:
for folder in p_list:
    ts_path = os.path.join(profile_path, folder)
    DB = DataBase(ts_path)
    DB.update()
    DB.loadfiles()


    pnames = list(DB.db.keys()) 
    
    for i, file in enumerate(pnames):

        if get_dir_size(consumption_path) < 1.5:
            consumption_gen(i, file, DB)

        else:
            create_new_folder(consumption_path)
            transtion_files(consumption_path)
            consumption_gen(i, file, DB)
        
        mobility_file_path = os.path.join(ts_path, f'{file}.pickle')
        
        used_profile = os.path.join(home_path, "db", "used", "profile")
        if get_dir_size(used_profile) < 1.5:
            pass

        else:
            create_new_folder(used_profile)
            transtion_files(used_profile)
            
        shutil.move(mobility_file_path, 
                    create_new_folder("not_needed", profile_consumption_availability = "profile", used_dir=True)
                   )
    
    DB = DataBase(consumption_path)
    DB.update()             
transtion_files(used_profile)
transtion_files(consumption_path)

: 

mname = list(DB_profile.db.keys())     # getting the id of the first mobility profile
HI = HeatInsulation(True)            # Creating the heat insulation by copying the default configuration
BEVS = BEVspecs()                    # Database that contains BEV models
#BEVS.show_models()

models = {
    "Tesla": 
        {"model": "Model 3 Long Range AWD",
         "year": 2019,
         "share": 22.73},
    "Volkswagen":
        {"model": "ID.3",
         "year": 2020,
         "share": 15.91},
    "Renault":
        {"model": "Zoe Q90",
         "year": 2017,
         "share": 15.91},
    "Hyundai":
        {"model": "Kona Electric 64 kWh",
         "year": 2019,
         "share": 11.63},
    "FIAT":
        {"model": "500e",
         "year": 2019,
         "share": 9.09},
    "BMW":
        {"model": "i3 33 kWh",
         "year": 2017,
         "share": 6.82},
    "MINI":
        {"model": "Cooper SE Level I",
         "year": 2020,
         "share": 6.82},
    "Audi": 
        {"model": "e-tron 55 quattro",
         "year": 2019,
         "share": 4.55},
    "Hyundai": 
        {"model": "IONIQ Electric 28 kWh",
         "year": 2019,
         "share": 4.55},
    "KIA": 
        {"model": "e-Niro 64 kWh",
         "year": 2019,
         "share": 2.27}
    
}

back_up_model = {
    "brand": "Volkswagen",
    "model": "e-Golf SE",
    "year": 2019
}

models_assigned = {}

i = 0
for brand in models.keys():
    models[brand]["number"] = round(number_profiles*models[brand]["share"]/100)
    #print(brand + " " +str(models[brand]["number"]))
    for k in range(0, round(models[brand]["number"])): 
        print(str(i))
        models_assigned[str(i)] = BEVS.model((
            brand, models[brand]["model"], models[brand]["year"]
        ))
        i += 1
while i <= number_profiles:
    
    models_assigned[str(i)] = BEVS.model((
            back_up_model["brand"], back_up_model["model"], back_up_model["year"]
        ))
    i += 1
    
    #plausibility check
len(models_assigned) - number_profiles

for i, file in enumerate(mname):
    car_model = models_assigned[str(i)]
    print(car_model.parameters['EV Model'])
    c = Consumption(file, car_model)
    c.load_setting_mobility(DB_profile)
    c.run(
        heat_insulation=HI,
        weather_country='DE',
        weather_year=2016,
        passenger_mass=75,                   # kg
        passenger_sensible_heat=70,          # W
        passenger_nr=1.5,                    # Passengers per vehicle including driver
        air_cabin_heat_transfer_coef=20,     # W/(m2K). Interior walls
        air_flow = 0.02,                     # m3/s. Ventilation
        driving_cycle_type='WLTC',           # Two options "WLTC" or "EPA"
        road_type=0,                         # For rolling resistance, Zero represents a new road.
        road_slope=0
    )
    c.save_profile(consumption_path)          

------------------------------
c) See some attributes

------------------------------

In [ ]:
#c.name
#c.input
#c.brand, c.model, c.year
#c.timeseries # Consumption in kWh/timestep -> timestep 15 min in this example
#c.profile.head()

: 

-------------------------
d) Visualization

-------------------------

In [ ]:
#fig_consumption = PLT.sankey(c.name, include=None, to_html=False, path=None)
#fig_consumption

: 

-------------------------------------------------------------------
## Step 3: Grid availability time series
-------------------------------------------------------------------
a) probability distribution and power rating for charging stations

-------------------------------------------------------------------

In [ ]:
from emobpy import Availability

: 

In [ ]:
station_distribution = {                  # Dictionary with charging stations type probability distribution per the purpose of the trip (location or destination)
    'prob_charging_point': {
        'errands': {'public': 0.5, 'none': 0.5},
        'escort': {'public': 0.5, 'none': 0.5},
        'leisure': {'public': 0.5, 'none': 0.5},
        'shopping': {'public': 0.5, 'none': 0.5},
        'home': {'public': 0.4, 'home': 0.6},
        'workplace': {'public': 0.0, 'workplace': 1.0, 'none': 0.0},   # If the vehicle is at the workplace, it will always find a charging station available (assumption)
        'driving': {'none': 0.99, 'fast75': 0.005, 'fast150': 0.005}}, # with the low probability given to fast charging is to ensure fast charging only for very long trips (assumption)
    'capacity_charging_point': {                                       # Nominal power rating of charging station in kW
        'public': 22,
        'home': 3.7,
        'workplace': 11,
        'none': 0,  # dummy station
        'fast75': 75,
        'fast150': 150}
}

: 

In [ ]:
c_list = os.listdir(consumption_path)
if ".DS_Store" in c_list:
    c_list.remove(".DS_Store")

: 

In [ ]:
def availability_gen(file, DB):
    ga = Availability(file, DB)
    ga.set_scenario(station_distribution)
    ga.run()
    ga.save_profile(availability_path)

: 

In [ ]:
for folder in c_list:
    ts_path = os.path.join(consumption_path, folder)
    DB = DataBase(ts_path)
    DB.update()
    DB.loadfiles()
    
    cnames = list(DB.db.keys()) 
    
    for file in cnames:

        if get_dir_size(availability_path) < 1.5:
            availability_gen(file, DB)

        else:
            create_new_folder(availability_path)
            transtion_files(availability_path)
            availability_gen(file, DB)
    
        cons_file_path = os.path.join(ts_path, f'{file}.pickle')
        
        used_consumption = os.path.join(home_path, "db", "used", "consumption")
        if get_dir_size(used_consumption) < 1.5:
            pass

        else:
            create_new_folder(used_consumption)
            transtion_files(used_consumption)
            
        shutil.move(cons_file_path, 
                        create_new_folder("not_needed", profile_consumption_availability = "consumption", used_dir=True)
                       )
    DB = DataBase(availability_path)
    DB.update()             
transtion_files(used_consumption)
transtion_files(availability_path)

: 

--------------------------------
b) See some attributes

--------------------------------

In [ ]:
#ga.name
#ga.input
#ga.battery_capacity
#ga.charging_eff
#ga.discharging_eff
#ga.soc_init
#ga.soc_min
#ga.timeseries

: 

# Export

In [ ]:
c_list = os.listdir(used_consumption)
if ".DS_Store" in c_list:
    c_list.remove(".DS_Store")
    
consumption_df = pd.DataFrame() 

for folder in c_list:
    DB = DataBase(os.path.join(used_consumption, folder))
    DB.update()
    DB.loadfiles()
    c_names = list(DB.db.keys()) 
    print(len(c_names))
    
    #get consumption_ts, name
    for c_file in c_names:
        profile = DB.db[c_file]
        try:
            col_name = profile["brand"]+"/" +profile["model"]+"/" +profile["year"]
            consumption = profile["timeseries"]["consumption"].to_frame().rename(
                columns={"consumption":col_name})
            consumption_df = pd.concat([consumption_df, consumption], axis=1)
            print(col_name)
        except KeyError:
            print(c_file)
            pass

: 

In [ ]:
availability_df = pd.DataFrame()
car_properties_df = pd.DataFrame(index = ["soc_end", "soc_init", "battery_capacity"])

ava_list = os.listdir(availability_path)
if ".DS_Store" in ava_list:
    ava_list.remove(".DS_Store")

for folder in ava_list:
    DB = DataBase(os.path.join(availability_path, folder))
    DB.update()
    DB.loadfiles()
    ava_names = list(DB.db.keys()) 
    print(len(ava_names))

    #charging cap [kW] 
    for a_file in ava_names:
        profile = DB.db[a_file]
        try:
            col_name = "_".join(profile["name"].split("_")[:-3])

            charging_eff = profile["charging_eff"]
            availability = (charging_eff*profile["timeseries"]["charging_cap"]
                           ).to_frame().rename(
                columns={"charging_cap":col_name})
            availability_df = pd.concat([availability_df, availability], axis=1)

            car_properties_df[col_name] = [
                profile["soc_end"], profile["soc_init"], profile["battery_capacity"]
            print(col_name)                  ]   
        except KeyError:
            print(a_file)
            pass

: 

In [ ]:
consumption_pickle = os.path.join(data_path, "consumption_new")
availaibilty_pickle = os.path.join(data_path, "availability_new")
property_pickle = os.path.join(data_path, "properties_new")

consumption_df.to_pickle(consumption_pickle)
availability_df.to_pickle(availaibilty_pickle)
car_properties_df.to_pickle(property_pickle)


: 